<a href="https://colab.research.google.com/github/atalsooraj/Multi-channel-Marketing-Spend-Optimization-using-Deep-Learning/blob/main/Data_Pre_processing_and_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
data_file = '/content/drive/MyDrive/606 Capstone/criteo_attribution_dataset.tsv.gz'

In [2]:
import pandas as pd
# df_Criteo_Attribution = pd.read_csv(DATA_FILE, sep='\t', compression='gzip')

df0 = pd.read_csv(data_file, sep='\t', compression='gzip')

KeyboardInterrupt: ignored

In [ ]:
df0.head()

In [ ]:
# df0[df0['uid']==7306395]

In [ ]:
len(df0)

In [ ]:
df0['campaign'].nunique()

In [ ]:
df0['conversion'].value_counts()

In [ ]:
df = df0.copy()

In [ ]:
del df0

Step 1: Creating a new column 'jid' by concatenating uid and conversion id. We do this becasue we want to analyze entire customer journeys i.e sequence of events.

In [ ]:
#Concatenating uid and conversion_id since we want to analyze entire customer journeys(sequence of events)

df['jid']= df['uid'].map(str)+ '_'+df['conversion_id'].map(str)

Step 2: Scaling the timestamp column.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Scaling timestamp and time_since_last_click columns 
minmaxscaler= MinMaxScaler()
for column in ('timestamp','time_since_last_click'):
  x= df[column].values.reshape(-1,1)
  df[column +'_norm']= minmaxscaler.fit_transform(x)


In [ ]:
df['campaign'].nunique()

Step 3: Randomly sampling 200 campaigns.

In [ ]:
#Randomly sampling 200 campaigns
import numpy as np
camapaigns_shortlisted= np.random.choice(df['campaign'].unique(),200,replace=False)
camapaigns_shortlisted[:5]

Finding the set difference between original and shortlisted campaign id's. the result will give unwanted campaign id's.

In [ ]:
not_reqd= np.setdiff1d(df['campaign'].values, camapaigns_shortlisted)


Considering all campaign id's except those present in not required array.

In [ ]:
df = df[~df['campaign'].isin(not_reqd)]

In [ ]:
len(df)

In [ ]:
df['campaign'].nunique()

In [ ]:
# df_sampled= df[df['campaign'][j for j in range(len(df['campaign']))]== [i for i in camapaigns_shortlisted]]

# df_sampled= df.loc[df['campaign'].apply(lambda x: x in camapaigns_shortlisted)]

#Source-https://stackoverflow.com/questions/21738882/fast-pandas-filtering

# df_sampled= df.loc[np.in1d(df['campaign'], camapaigns_shortlisted)]

Step 4: Counting the instances of each 'jid' and only considering theose journey id's having more than 1 occurence.

In [ ]:
grouped= df.groupby(['jid'])['uid'].count().reset_index(name='count')

In [ ]:
#grouped.head()

In [ ]:
df2= df[df['jid'].isin(grouped[grouped['count'] >= 2]['jid'].values )]  #Only considering journey id's with more than 1 event. Here, we are indirectly removing jid's with only 1 occurence.

In [ ]:
#df2.head()

In [ ]:
del df
del not_reqd


In [ ]:
len(df2)

In [ ]:
df_0= df2[df2['conversion']==0]
df_1= df2[df2['conversion']==1]

In [ ]:
len(df_0), len(df_1)

In [ ]:
del df2

Step 5: Balancing the dataset: The number of convertd and non-converted events should be equal so that our model can have a balanced learning. We take all converted journeys and iteratively add non-converted events until the dataset is balanced. We do it this way because we want to balance the number of events but also want the whole journey.

In [ ]:
df_0_jids= np.array_split(df_0['jid'].unique(),100* df_0.shape[0]/df_1.shape[0])

In [ ]:
df_0_sampled = pd.DataFrame(data=None, columns=df_0.columns)

In [ ]:
for array in df_0_jids:
  df_0_sampled = pd.concat([df_0_sampled, df_0[df_0.jid.isin(array)]])

  if df_0_sampled.shape[0]> df_1.shape[0]:
    break


In [ ]:
df_final= pd.concat([df_0_sampled, df_1]).sample(frac=1).reset_index(drop=True)

In [ ]:
del df_0,df_1,df_0_jids,df_0_sampled

In [ ]:
df_final['conversion'].value_counts()

In [ ]:
df_final.head()

In [ ]:
len(df_final)

Finding unique values in each categorical column. Here, cat 7 has way too many features so we are going to ignore this columns and use all the others.

In [ ]:
df_final[df_final.columns[13:22]].nunique()

Creating a dictionary with all unique categorical variables as keys and assigning them values based on length of the dictionary when they are added.

In [ ]:
mapper= {}
col_names= ['cat1','cat2','cat3','cat4','cat5','cat6','cat8','cat9']
for i, col_name in enumerate(col_names):
  for val in df_final[col_name].unique():
    mapper[val*10 +i]=len(mapper)


Creating a new column containing the list of all the categories present in that corresponding rows.

In [ ]:
df_ext = df_final.copy()
df_ext['cats']= df_ext[col_names].values.tolist()

In [ ]:
df_ext.head()

Creating a function to do one hot encoding for each row value in campaign's column and update the encoded value in an array.

In [ ]:
def one_hot(values):
  v= np.zeros(len(mapper))
  for i, val in enumerate(values):
    mapped_val_id= mapper[val*10+i]
    v[mapped_val_id]= 1
  return v

In [ ]:
import numpy as np

In [ ]:
df_ext['cats']= df_ext['cats'].map(one_hot)

In [ ]:
#df_ext['cats'][0]

Getting all unique values in the campaign column and creating a dictionary with keys as campaign id and values as the length of the dictionary.

In [ ]:
mapper= {}
col_names= ['campaign']
for i, col_name in enumerate(col_names):
  for val in df_ext[col_name].unique():
    mapper[val*10 +i]=len(mapper)


df_ext['campaigns']= df_ext[col_names].values.tolist()



In [ ]:
df_ext.head()

using the one-hot function for campaign column.

In [ ]:
df_ext['campaigns']= df_ext['campaigns'].map(one_hot)

Sorting the obtained dataframe by normalized timestamp value.

In [ ]:
df_model= df_ext.sort_values(by=['timestamp_norm'])

In [ ]:
df_model.head()

In [ ]:
# df_model['campaigns'][94389]

In [ ]:
journey_counts= df_model.groupby(['jid'])['uid'].count().reset_index(name='count').groupby(['count']).count()
counts= journey_counts.index
no_of_jids= journey_counts.values 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(counts,no_of_jids)
plt.ylim(0,20)
plt.xlim(0,200)
plt.xlabel('Journey length (number of touchpoints)')
plt.ylabel('No. of customers')
plt.show()

From the above plot, it is clear that as the journey lenght increases, the number of jid's is decreasing. This is expected and makes sense. This re-affirms that we can develop a model for sequential data.

Counting the number of events for each campaign_id

In [ ]:
counters= np.zeros(200)
for campaign_one_hot in df_model['campaigns'].values:
  campaign_id= np.argmax(campaign_one_hot)
  counters[campaign_id]= counters[campaign_id]+1


In [ ]:
# counters

To understand what counters represents, let us consider the first row from df_modelhaving an index 94389. When we apply the argmax function to an encoded array like in our case to the campaigns array, we get the position of value 1 in the encoded campaigns array i.e 86 here. Intitially, counters[86] will be 0(since we added 200 zeros). When a campaign's value in the encoded array is 86, it will increase the value of counter[86] by 1 and hence giving us the total number of occurences of a campaign in the entire dataset. So, now if we check the value of counters[86] we will get the count of the campaign 884761 which is 1368. So there are 1368 instances of the campaign 884761.

In [ ]:
np.argmax(df_model['campaigns'][94389])

In [ ]:
counters[86]

In [ ]:
df_converted= df_model[df_model['conversion']==1]

In [ ]:
# df_converted.head()

In [ ]:
len(df_converted)

In [ ]:
df_converted['jid'].nunique()

Counting the number of times a given campaign is the last touch before the conversion. here, when we groupby jid and timestamp_norm, we get 2 columns. One column is jid containing jid's  and the other containing the corresponding timestamp_norm value for the event. There may be multiple instances of jid's having multiple values of timestamp_norm values. The transform(max) function gives us the maximum value of timestamp_norm from among this sample of jid's. Finally, we check if this obtained max value is equal to the value in timestamp_norm column i.e if this was the last event and return a boolean value of either true or False along with the index value of timestamp_norm.

In [ ]:
index= df_converted.groupby(['jid'])['timestamp_norm'].transform(max) == df_converted['timestamp_norm']

In [ ]:
index[:5]

In [ ]:
len(index.tolist())

Taking all last touch arrays from campaigns column based on index of True values.

In [ ]:
# df_converted[index]['campaigns']

As done before, counting the number of times the event was last touch before conversion.

In [ ]:
counters_2= np.zeros(200)
for campaign_one_hot in df_converted[index]['campaigns'].values:
  campaign_id= np.argmax(campaign_one_hot)
  counters_2[campaign_id]= counters_2[campaign_id]+1

Now, we have obtained the number of occurences for a campaign (counters) and number of times that campaign was last touch(counters). The ratio between the number of journeys in which a given campaign is the last event and the total number of events for the same campaign gives us the attribution weight (which is return per impression).

In [ ]:
attributions= counters_2/counters

In [ ]:
campaign_idx= range(0,200)
fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(111)
plt.bar( range(len(attributions[campaign_idx])), attributions[campaign_idx], label='LTA' )
plt.xlabel('Sampled 200 campaigns')
plt.ylabel('Return per impression (attribution)')

In [ ]:
del camapaigns_shortlisted, data_file, df_converted, df_ext, df_final, index, mapper, minmaxscaler, one_hot

In [ ]:
del attributions

In [ ]:
 del campaign_id,campaign_idx, campaign_one_hot, col_name, col_names, column, counters, counters_2, counts,grouped, i, journey_counts, no_of_jids

In [ ]:
df_model.to_csv('Dataframe_test.csv')

In [ ]:
# def features_for_logistic_regression(df):

#     def pairwise_max(series):
#         return np.max(series.tolist(), axis = 0).tolist()
    
#     aggregation = {                      # aggregation specification for each feature
#         'campaigns': pairwise_max,
#         'cats': pairwise_max,
#         'click': 'sum',
#         'cost': 'sum',
#         'conversion': 'max'
#     }
    
#     df_agg = df.groupby(['jid']).agg(aggregation)
    
#     df_agg['features'] = df_agg[['campaigns', 'cats', 'click', 'cost']].values.tolist()
    
#     return (
#         np.stack(df_agg['features'].map(lambda x: np.hstack(x)).values),
#         df_agg['conversion'].values
#     )



In [ ]:
# x,y = features_for_logistic_regression(df_model)
# print(np.shape(x))

In [ ]:
# from sklearn.model_selection import train_test_split